## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dunedin,NZ,-45.8742,170.5036,42.80,65,97,17.00,overcast clouds
1,1,Zetea,RO,46.3833,25.3667,51.46,88,45,4.21,light rain
2,2,Adrar,MR,20.5022,-10.0711,93.76,25,63,9.40,broken clouds
3,3,Gusau,NG,12.1628,6.6614,69.73,88,73,15.17,moderate rain
4,4,Saldanha,ZA,-33.0117,17.9442,50.05,75,84,2.68,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Ko Samui,TH,9.5357,99.9357,80.51,83,75,6.91,moderate rain
12,12,Sinnamary,GF,5.3833,-52.9500,80.40,82,87,15.10,overcast clouds
13,13,Hilo,US,19.7297,-155.0900,88.29,70,75,7.00,broken clouds
14,14,Pochutla,MX,15.7432,-96.4661,76.15,84,100,4.03,overcast clouds
25,25,Namie,JP,37.4833,141.0000,83.71,95,82,12.91,broken clouds
29,29,Najran,SA,17.4924,44.1277,77.65,19,76,2.84,broken clouds
31,31,Avarua,CK,-21.2078,-159.7750,75.25,94,100,10.36,light rain
35,35,Haimen,CN,23.1935,116.6142,80.49,70,84,3.58,broken clouds
38,38,Butaritari,KI,3.0707,172.7902,81.75,78,84,16.13,light rain
48,48,Kapaa,US,22.0752,-159.3190,84.52,61,0,11.50,clear sky


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                169
City                   169
Country                169
Lat                    169
Lng                    169
Max Temp               169
Humidity               169
Cloudiness             169
Wind Speed             169
Current Description    169
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Ko Samui,TH,80.51,moderate rain,9.5357,99.9357,
12,Sinnamary,GF,80.40,overcast clouds,5.3833,-52.9500,
13,Hilo,US,88.29,broken clouds,19.7297,-155.0900,
14,Pochutla,MX,76.15,overcast clouds,15.7432,-96.4661,
25,Namie,JP,83.71,broken clouds,37.4833,141.0000,
29,Najran,SA,77.65,broken clouds,17.4924,44.1277,
31,Avarua,CK,75.25,light rain,-21.2078,-159.7750,
35,Haimen,CN,80.49,broken clouds,23.1935,116.6142,
38,Butaritari,KI,81.75,light rain,3.0707,172.7902,
48,Kapaa,US,84.52,clear sky,22.0752,-159.3190,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Ko Samui,TH,80.51,moderate rain,9.5357,99.9357,The Passage
12,Sinnamary,GF,80.40,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
13,Hilo,US,88.29,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
14,Pochutla,MX,76.15,overcast clouds,15.7432,-96.4661,Hotel Posada San Jose
25,Namie,JP,83.71,broken clouds,37.4833,141.0000,いこいの村なみえ
...,...,...,...,...,...,...,...
549,Makakilo City,US,85.57,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
550,Abu Dhabi,AE,89.71,clear sky,24.4667,54.3667,Beach Rotana
555,Puerto Escondido,MX,79.63,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
557,Jacksonville Beach,US,86.27,overcast clouds,30.2947,-81.3931,Courtyard by Marriott Jacksonville Beach Ocean...


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))